In [72]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]

print(y.shape)


(891, 1)


In [79]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNovos(BaseEstimator, TransformerMixin):
    def __init__(self, incluirFamilySize=True, incluirIsAlone=True):
        self.incluirFamilySize = incluirFamilySize
        self.incluirIsAlone = incluirIsAlone
    def fit(self, X, y=None):        
        return self
    def transform(self, X, y=None):
        data = X.copy()

        if self.incluirFamilySize:
            data['FamilySize'] = data['SibSp'] + data['Parch']

            # if self.incluirIsAlone:
            #     data['IsAlone'] = data['FamilySize'] == 0
            #     data['IsAlone'] = data['IsAlone'].astype(int)

        return data

atributosNovos = AtributosNovos()
Xnovos = atributosNovos.fit_transform(X)

print(Xnovos)

     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
..           ...     ...                                                ...   
886          887       2                              Montvila, Rev. Juozas   
887          888       1                       Graham, Miss. Margaret Edith   
888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
889          890       1                              Behr, Mr. Karl Howell   
890          891       3                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket  

In [89]:
from sklearn.base import BaseEstimator, TransformerMixin

pronomes = {
            'Mr':"Mr",
            'Mrs': "Mrs",
            'Ms': "Mrs",
            'Mme':"Mrs",
            'Mlle':"Miss",
            'Miss':"Miss",
            'Master':"Master",
            'Dr':"Dr",
            'Rev':"Officer",
            'Col':"Officer",
            'Capt':"Officer",
            'Major':"Officer",
            'Lady':"Royalty",
            'Sir':"Royalty",
            'the Countess':"Royalty",
            'Dona':"Royalty",
            'Don':"Royalty",
            'Jonkheer':"Royalty" 
        }

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

def tratarPronome(pronome):
    return pronomes[pronome]

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True, tratarPronome=True):
        self.excluirName = excluirName
        self.tratarPronome = tratarPronome
    def fit(self, X, y=None):
        self.colunasIndesejadas = ["PassengerId", "Ticket", "Cabin", "Embarked"]
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        data = X.copy()

        Xdrop = data.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
            if self.tratarPronome:
                Xdrop['Name'] = Xdrop['Name'].apply(tratarPronome)

        return Xdrop

atributosDesejados = AtributosDesejados(excluirName=False)
Xdesejados = atributosDesejados.fit_transform(Xnovos)

Pclass          0
Name            0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
FamilySize      0
dtype: int64
Pclass          int64
Name           object
Sex            object
Age           float64
SibSp           int64
Parch           int64
Fare          float64
FamilySize      int64
dtype: object


In [75]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        data = X.copy()

        return data[self.colunasNumericas].to_numpy()

atributosNumericos = AtributosNumericos()
Xnumericos = atributosNumericos.fit_transform(Xdesejados)
print(atributosNumericos.colunasNumericas)
print(Xnumericos.isnull().sum())


Index(['Pclass', 'Age', 'Fare', 'FamilySize', 'IsAlone'], dtype='object')


In [76]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        data = X.copy()

        return data[self.colunasCategoricas].to_numpy()


In [77]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosNovos', AtributosNovos()),
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {    
    'atributosDesejados__excluirName': [True, False],
    'classificador__max_depth': [None] + list(range(3,9,2))
    #'classificador__max_depth': [5]
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, scoring='roc_auc')

scores = cross_validate(modelo, X, y.values.ravel(), cv=RepeatedKFold())

print(f"scores={scores['test_score']}, média={np.mean(scores['test_score'])}, desv_pad={np.std(scores['test_score'])}")

scores=[0.83245025 0.89799862 0.86869706 0.86140231 0.85628898 0.86807279
 0.87376645 0.87814465 0.85320388 0.83546071 0.82214941 0.84744724
 0.87253278 0.84755382 0.89936817 0.90816456 0.81350007 0.86842792
 0.86618357 0.86871469 0.87929698 0.88893371 0.83128307 0.83291925
 0.87249036 0.86138996 0.87562189 0.86622921 0.86697052 0.85120321
 0.86311001 0.87554846 0.90588235 0.84252646 0.86183953 0.9136163
 0.79147013 0.87846045 0.84316038 0.86559794 0.83207716 0.84960333
 0.86513941 0.89186508 0.86604278 0.86675579 0.84913571 0.91293316
 0.89827659 0.80235619], média=0.8628252661175424, desv_pad=0.026530271503137554


In [95]:
modelo.fit(X,y.values.ravel())
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)

c:\Users\leandropo\_git\ifes\rpam-regressao\.venv\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\leandropo\_git\ifes\rpam-regressao\.venv\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\leandropo\_git\ifes\rpam-regressao\.venv\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\leandropo\_git\ifes\rpam-regressao\.venv\lib\site-packages\sklearn\pip